# Intro to SQL: Restaurant Case Study

In this notebook, I explore basic queries like inserts, updates, and select statements by pouring over hypothetical restaurant data.
**this notebook uses a pgsql database in a vm and cannot be run locally without standing up an EC2 cluster.**

In [2]:
import math
import numpy as np
import pandas as pd
import psycopg2

In [3]:
# borrowing a function from stackoverflow to easily read sqlite queries
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    for column in df:
        if df[column].dtype == "float64":
            fraction_flag = False
            for value in df[column].values:
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True
            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    return(df)

In [ ]:
connection = psycopg2.connect(
    user = "postgres",
    password = "****",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# Total sales as a dollar amount, total number of sales, average dollar amount per sale


In [ ]:
rollback_before_flag = True
rollback_after_flag = True
query = """
SELECT 
    SUM(total_amount) as total_sales_dollars,  
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    COUNT(sale_id) as total_number_of_sales,
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
FROM Sales 
"""
df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# Total sales as a dollar amount, total number of sales, average dollar amount per sale by store

In [ ]:
query = """
SELECT 
    
    city as store_name,
    SUM(total_amount) as total_sales_dollars,  
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    COUNT(sale_id) as total_number_of_sales,
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
    
FROM 
Sales S  INNER JOIN Stores E
on S.store_id = E.store_id
GROUP BY store_name 
ORDER BY store_name ASC 
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# Total sales as a dollar amount, total number of sales, average dollar amount per sale by month

In [ ]:
query = """

SELECT 
    
    EXTRACT(month FROM sale_date) as month_number,
    
    TO_CHAR(sale_date::date, 'Month') AS month, 
    
    SUM(total_amount) as total_sales_dollars,  
    
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    
    COUNT(sale_id) as total_number_of_sales,
    
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
    
    
FROM Sales 

GROUP BY 1,2


"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Total sales as a dollar amount, total number of sales, average dollar amount per sale by store and month

In [ ]:
query = """

SELECT 

    city as store_name,
    EXTRACT(month FROM sale_date) as month_number,
    TO_CHAR(sale_date::date, 'Month') AS month,
    SUM(total_amount) as total_sales_dollars,  
    
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    COUNT(sale_id) as total_number_of_sales,
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
    
    
FROM 
Sales S  
INNER JOIN Stores E on S.store_id = E.store_id
GROUP BY 1,2,3
ORDER BY 1 ASC, 2 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Total sales as a dollar amount, total number of sales, average dollar amount per sale by day of week

In [ ]:
query = """

SELECT 
    EXTRACT(dow FROM sale_date) as dow,
    TO_CHAR(sale_date::date, 'Day') AS day_of_week,
    SUM(total_amount) as total_sales_dollars,
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    COUNT(sale_id) as total_number_of_sales,
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
FROM Sales 

GROUP BY 1,2
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Total sales as a dollar amount, total number of sales, average dollar amount per sale by store and day of week

In [ ]:
query = """

SELECT 

    city as store_name,
    EXTRACT(dow FROM sale_date) as dow,
    TO_CHAR(sale_date::date, 'Day') AS day_of_week, 
    SUM(total_amount) as total_sales_dollars,  
    round(SUM(total_amount)/1000000,1) as total_sales_million_dollars,
    COUNT(sale_id) as total_number_of_sales,
    round(AVG(total_amount),2) as average_dollar_amount_per_sale
    
    
FROM 

Sales S INNER JOIN Stores E
on S.store_id = E.store_id
GROUP BY 1,2,3
ORDER BY 1 ASC, 2 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Total Number of Customers for all of AGM

In [ ]:
query = """
SELECT
    COUNT(*) AS total_number_of_customers
FROM CUSTOMERS
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Customers by Store

In [ ]:
query = """
SELECT
    S.city as store_name,
    COUNT(*) AS total_number_of_customers
FROM Customers C 
INNER JOIN Stores S ON C.closest_store_id = S.store_id
GROUP BY 1 
ORDER BY 1 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Signed up, Not Bought Anything

In [ ]:
query = """

SELECT 
    last_name,
    first_name
    
FROM Customers C
LEFT JOIN Sales S on C.customer_id = S.customer_id
WHERE S.customer_id is NULL
ORDER BY 1 ASC, 2 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Customers as % of Pop by Zip Code

In [ ]:
query = """
SELECT
    C.zip as zip,
    ROUND((count(C.customer_id)/max(Z.population))*100, 3) as percentage_customers_per_population
FROM Customers C
INNER JOIN Zip_codes Z on C.zip = Z.zip
GROUP BY 1
ORDER BY count(C.customer_id)/max(Z.population) DESC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Meal Purchases

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT 
    
    sum(quantity) as total_meals_purchased

FROM line_items 

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Meal Purchases over Month

In [ ]:
query = """

SELECT 
    EXTRACT(month FROM sale_date) as month_number,
    TO_CHAR(sale_date::date, 'Month') AS month,
    SUM(quantity) as total_meals_purchased
FROM Sales S
INNER JOIN Line_items L on S.sale_id = L.sale_id AND S.store_id = L.store_id
GROUP BY 1,2
ORDER BY 1 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

# Meal Purchases by Day + Week

In [ ]:
query = """
SELECT 
    EXTRACT(dow FROM sale_date) as dow,
    TO_CHAR(sale_date::date, 'Day') AS day_of_week,
    P.description as meal_name,
    SUM(quantity) as total_meals_purchased
FROM

Sales S
INNER JOIN Line_items L on S.sale_id = L.sale_id AND S.store_id = L.store_id
INNER JOIN Products P on L.product_id = P.product_id

GROUP BY 1,2,3
ORDER BY 1 ASC, 3 ASC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df